# Evaluate ALL features

Similar to notebook 3 we package everything inside a for loop to evaluate all the features.

In [1]:
# TO REMOVE when notebook is stable

%load_ext autoreload
%autoreload 2

### Common Imports

In [2]:
import numpy
import torch
import seaborn
import tarfile
import os
import matplotlib
import matplotlib.pyplot as plt
from anndata import read_h5ad

# tissue_purifier import
import tissue_purifier as tp

### Download the annotated anndata object 

Altenatively you can use the anndata files generated by running notebook2_all.

In [3]:
import tissue_purifier.io

bucket_name = "ld-data-bucket"
annotated_anndata_source_path = "tissue-purifier/annotated_slideseq_testis_anndata_h5ad.tar.gz"
annotated_anndata_dest_path = "./annotated_slideseq_testis_anndata_h5ad.tar.gz"
annotated_anndata_dest_folder = "./testis_anndata_annotated"

#tp.io.download_from_bucket(bucket_name, annotated_anndata_source_path, annotated_anndata_dest_path)   
#with tarfile.open(annotated_anndata_dest_path, "r:gz") as fp:
#    fp.extractall(path=annotated_anndata_dest_folder)
    
# Make a list of all the h5ad files in the annotated_anndata_dest_folder
fname_list = []
for f in os.listdir(annotated_anndata_dest_folder):
    if f.endswith('.h5ad'):
        fname_list.append(f)
print(fname_list)

['anndata_sick3.h5ad', 'anndata_sick1.h5ad', 'anndata_sick2.h5ad', 'anndata_wt2.h5ad', 'anndata_wt1.h5ad', 'anndata_wt3.h5ad']


### Decide how to filter the anndata object

In [4]:
# filter cells parameters
fc_bc_min_umi = 200                  # filter cells with too few UMI
fc_bc_max_umi = 3000                 # filter cells with too many UMI
fc_bc_min_n_genes_by_counts = 10     # filter cells with too few GENES
fc_bc_max_n_genes_by_counts = 2500   # filter cells with too many GENES
fc_bc_max_pct_counts_mt = 5          # filter cells with mitocrondial fraction too high

# filter genes parameters
fg_bc_min_cells_by_counts = 3000      # filter genes which appear in too few CELLS

# filter rare cell types parameters
fctype_bc_min_cells_absolute = 100   # filter cell-types which are too RARE in absolute number
fctype_bc_min_cells_frequency = 0.01 # filter cell-types which are too RARE in relative abundance

### Open the first annotated anndata 

In [5]:
adata = read_h5ad(filename=os.path.join(annotated_anndata_dest_folder, fname_list[0]))
adata

AnnData object with n_obs × n_vars = 33441 × 23514
    obs: 'x', 'y', 'cell_type'
    obsm: 'barlow', 'dino', 'ncv_k10', 'ncv_k100', 'ncv_k20', 'ncv_k200', 'ncv_k50', 'ncv_k500', 'simclr', 'vae'

### compute few metrics

In [6]:
import scanpy as sc
cell_type_key = "cell_type"

# mitocondria metrics
adata.var['mt'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

# counts cells frequency
tmp = adata.obs[cell_type_key].values.describe()
print(tmp)
mask1 = (tmp["counts"] > fctype_bc_min_cells_absolute)
mask2 = (tmp["freqs"] > fctype_bc_min_cells_frequency)
mask = mask1 * mask2
cell_type_keep = set(tmp[mask].index.values)
adata.obs["keep_ctype"] = adata.obs["cell_type"].apply(lambda x: x in cell_type_keep)

# Note that adata has extra annotation now
adata

             counts     freqs
categories                   
ES            12552  0.375348
Endothelial     417  0.012470
Leydig          340  0.010167
Macrophage      623  0.018630
Myoid           969  0.028976
RS             6780  0.202745
SPC            8069  0.241291
SPG            2238  0.066924
Sertoli        1453  0.043450


AnnData object with n_obs × n_vars = 33441 × 23514
    obs: 'x', 'y', 'cell_type', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'keep_ctype'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    obsm: 'barlow', 'dino', 'ncv_k10', 'ncv_k100', 'ncv_k20', 'ncv_k200', 'ncv_k50', 'ncv_k500', 'simclr', 'vae'

### Filter out cells, genes and cell-type

In [7]:
adata = adata[adata.obs["total_counts"] > fc_bc_min_umi, :] 
adata = adata[adata.obs["total_counts"] < fc_bc_max_umi, :] 
adata = adata[adata.obs["n_genes_by_counts"] > fc_bc_min_n_genes_by_counts, :] 
adata = adata[adata.obs["n_genes_by_counts"] < fc_bc_max_n_genes_by_counts, :] 
adata = adata[adata.obs["pct_counts_mt"] < fc_bc_max_pct_counts_mt, :]
adata = adata[adata.obs["keep_ctype"] == True, :]
adata = adata[:, adata.var["n_cells_by_counts"] > fg_bc_min_cells_by_counts]

# Loop to train multiple gene_regression models

In [8]:
from tissue_purifier.genex import *
gr_ckpt_dir = "gr_ckpt"
filename_no_covariate = os.path.join(gr_ckpt_dir, "gr_no_covariate.pt")

covariate_keys = ['barlow_pca', 'dino_pca', 'simclr_pca', 'barlow', 'dino', 'simclr']
# covariate_keys = ['ncv_k10', 'ncv_k100', 'ncv_k20', 'ncv_k200', 'ncv_k50', 'ncv_k500']
# covariate_keys = ['ncv_k100', 'ncv_k20', 'ncv_k200', 'ncv_k50', 'ncv_k500']

n_train_steps_from_scratch = 20000
n_train_steps_from_pretrained = 5000
# n_train_steps_from_scratch = 10
# n_train_steps_from_pretrained = 10

l1_strengths = [0.01] #, 0.1, 1.0]
l2_strengths = [] # [0.01, 0.1, 1.0]

gr = GeneRegression()
gr.configure_optimizer(optimizer_type='adam', lr=5E-3)

for n_iter, covariate_key in enumerate(covariate_keys):
    print(covariate_key)

    # decide the subsample size (to fit into GPU memory)
    if covariate_key.startswith("ncv") or covariate_key.endswith("_pca"):
        subsample_size_cells = 2000
    else:
        subsample_size_cells = 200 
    
    
    if covariate_key.endswith("_pca"):
        
        # make gene dataset with PCA
        gene_dataset = make_gene_dataset_from_anndata(
            anndata=adata,
            cell_type_key='cell_type',
            covariate_key=covariate_key.split("_")[0],
            preprocess_strategy='z_score',
            apply_pca=True,
            n_components=9,)
        
    else:
        
        # make the dataset without PCA
        gene_dataset = make_gene_dataset_from_anndata(
            anndata=adata,
            cell_type_key='cell_type',
            covariate_key=covariate_key,
            preprocess_strategy='raw',
            apply_pca=False)
    
    # split into train/test/val (note that we provide the random_state for reproducibility) 
    train_test_val_dataset = next(iter(train_test_val_split(gene_dataset, random_state=0)))
    filename_dataset = os.path.join(gr_ckpt_dir, "gr_{}_dataset.pt".format(covariate_key))
    torch.save(train_test_val_dataset, filename_dataset)
    train_dataset, test_dataset, val_dataset = train_test_val_dataset
    
    ## only at the very beginning train a model with no_covariance from scratch
    #if n_iter == 0:
    #    print("no covariate")
    #    gr.train(
    #        dataset=train_dataset,
    #        n_steps=n_train_steps_from_scratch,
    #        print_frequency=100,
    #        use_covariates=False,
    #        l1_regularization_strength=None,
    #        l2_regularization_strength=None,
    #        eps_range=(1.0E-5, 1.0E-2),
    #        subsample_size_cells=subsample_size_cells,
    #        subsample_size_genes=None,
    #        initialization_type="scratch")
    #    gr.save_ckpt(filename_no_covariate)
    #    print("saved file {}".format(filename_no_covariate))
    
    # train with no regularization start from pretrained model with no_covaraince. 
    # This will make training faster
    print("no regularizations")
    gr.partial_load_ckpt(filename=filename_no_covariate)
    gr.train(
        dataset=train_dataset,
        n_steps=n_train_steps_from_pretrained,
        print_frequency=100,
        use_covariates=True,
        l1_regularization_strength=None,
        l2_regularization_strength=None,
        eps_range=(1.0E-5, 1.0E-2),
        subsample_size_cells=subsample_size_cells,
        subsample_size_genes=None,
        initialization_type="pretrained")
    filename = os.path.join(gr_ckpt_dir, "gr_{}_no_regularization.pt".format(covariate_key))
    gr.save_ckpt(filename)
    print("saved file {}".format(filename))
        
    # train with l1 regularization.
    # start from pretrained model with no_covaraince. 
    # This will make training faster
    for l1 in l1_strengths: 
        gr.partial_load_ckpt(filename=filename_no_covariate)
        print("l1 regularization", l1)
        gr.train(
            dataset=train_dataset,
            n_steps=n_train_steps_from_pretrained,
            print_frequency=100,
            use_covariates=True,
            l1_regularization_strength=l1,
            l2_regularization_strength=None,
            eps_range=(1.0E-5, 1.0E-2),
            subsample_size_cells=subsample_size_cells,
            subsample_size_genes=None,
            initialization_type="pretrained")
        filename = os.path.join(gr_ckpt_dir, "gr_{}_l1_{}.pt".format(covariate_key, l1))
        gr.save_ckpt(filename)
        print("saved file {}".format(filename))
    
    # train with l1 regularization.
    # start from pretrained model with no_covaraince. 
    # This will make training faster
    for l2 in l2_strengths: 
        gr.partial_load_ckpt(filename=filename_no_covariate)
        print("l2 regularization", l2)
        gr.train(
            dataset=train_dataset,
            n_steps=n_train_steps_from_pretrained,
            print_frequency=100,
            use_covariates=True,
            l1_regularization_strength=None,
            l2_regularization_strength=l2,
            eps_range=(1.0E-5, 1.0E-2),
            subsample_size_cells=subsample_size_cells,
            subsample_size_genes=None,
            from_scratch=True)
        filename = os.path.join(gr_ckpt_dir, "gr_{}_l2_{}.pt".format(covariate_key, l2))
        gr.save_ckpt(filename)
        print("saved file {}".format(filename))

barlow_pca
no regularizations
training from pretrained model
[iter 1]  loss: 21282260.0000
[iter 100]  loss: 21593164.0000
[iter 200]  loss: 21511520.0000
[iter 300]  loss: 21511872.0000
[iter 400]  loss: 21744888.0000
[iter 500]  loss: 21766524.0000
[iter 600]  loss: 21555148.0000
[iter 700]  loss: 21516384.0000
[iter 800]  loss: 21813180.0000
[iter 900]  loss: 21350082.0000
[iter 1000]  loss: 21245302.0000
[iter 1100]  loss: 21284488.0000
[iter 1200]  loss: 21101572.0000
[iter 1300]  loss: 21653532.0000
[iter 1400]  loss: 20838996.0000
[iter 1500]  loss: 21221020.0000
[iter 1600]  loss: 21411156.0000
[iter 1700]  loss: 21481326.0000
[iter 1800]  loss: 21480472.0000
[iter 1900]  loss: 21494240.0000
[iter 2000]  loss: 21225648.0000
[iter 2100]  loss: 21219120.0000
[iter 2200]  loss: 21686632.0000
[iter 2300]  loss: 21728286.0000
[iter 2400]  loss: 21218912.0000
[iter 2500]  loss: 21428094.0000
[iter 2600]  loss: 21401528.0000
[iter 2700]  loss: 21463344.0000
[iter 2800]  loss: 21955412

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(10,4))

axes[0].plot(no_cov_loss)
axes[0].plot(barlow_loss)
axes[0].set_ylim(2E7, 2.5E7)

axes[1].plot(no_cov_loss)
axes[1].plot(barlow_loss)
axes[1].set_ylim(2E7, 2.5E7)
axes[1].set_yscale("log")
axes[1].set_xscale("log")

# Compare all regressions

1. check the loss function converged
2. for each covariate_key compute the ration Q_with_covariance vs Q_no_covariance to select the best regularization
3. compare across covariate_key

REMOVE q_empirical from predict b/c it is misleading
I don't like the fact that epsilon is clustered by cell_type. Is it because the model is un-identifiable (both beta0 and eps are K x G). Think about changing model to have eps only being gene dependent. (as by Dylan)